In [ ]:
from dotenv import load_dotenv

load_dotenv()


In [2]:
with open('anatomy.txt', 'r', encoding='utf-8')as f:
          file = f.read()

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 600,
          chunk_overlap = 0,
          length_function = len
)

chunks = text_splitter.split_text(file)

In [4]:
from langchain_community.vectorstores import AzureSearch
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document
from azure.core.exceptions import ServiceRequestError, ServiceResponseError

documentos = [Document(page_content=chunk) for chunk in chunks]

try:
    chunk_store = AzureSearch.from_documents(
        azure_search_endpoint='https://hackathon-storage.search.windows.net',
        azure_search_key='Q4pGUsJ9fWNNdHpkkEDve9rBXGIYz1DNWK9b7LENFAAzSeAP8Yqb',
        documents=documentos,
        embedding=OllamaEmbeddings(model='nomic-embed-text:latest')
    )
except (ServiceRequestError, ServiceResponseError) as e:
    print(f"Error de conexión con Azure Search: {e}")
    # Manejo alternativo, como usar un almacenamiento local

In [13]:
chunks_vector = chunk_store.as_retriever(search_type="hybrid")  # Configuración básica del retriever

# Pasa `k` directamente en la consulta
retrieved_docs = chunks_vector.invoke("what is MDT?", k=1)

# Imprime los resultados
print("Documentos recuperados:", retrieved_docs)


TypeError: langchain_community.vectorstores.azuresearch.AzureSearch.hybrid_search() got multiple values for keyword argument 'k'

In [11]:
retriever = chunk_store.as_retriever(search_kwargs={"k": 1})

# Realiza una consulta
query = "What is AI?"
retrieved_docs = retriever.invoke(query)

# Inspecciona los resultados
for doc in retrieved_docs:
    print(f"ID: {doc.metadata.get('id')}, Contenido: {doc.page_content}")



TypeError: langchain_community.vectorstores.azuresearch.AzureSearch.hybrid_search() got multiple values for keyword argument 'k'

In [11]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="tinyllama:1.1b",
    base_url="http://localhost:11434",
    temperature=0,
    format="json" 
)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

# Crea un prompt más explícito
prompt = ChatPromptTemplate.from_template(
    "Contexto: {context}\n\nPregunta: {question}\n\nResponde basándote en el contexto proporcionado:"
)

In [ ]:
retrieved_docs = chunks_vector.invoke('what is MDT?')
print("Documentos recuperados:", retrieved_docs)

# Prueba directa del modelo
try:
    response = llm.invoke(
        prompt.format(
            context=retrieved_docs, 
            question='what is MDT?'
        )
    )
    print("Respuesta del modelo:", response)
except Exception as e:
    print(f"Error al invocar el modelo: {e}")

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

semantic_rag_chain = (
          {'context':chunks_vector , 'question': RunnablePassthrough()}
          | prompt
          | llm
          | StrOutputParser()
)

In [ ]:
semantic_rag_chain.invoke('what is MDT? ')

In [10]:
# Método alternativo de invocación
from langchain_core.messages import HumanMessage

response = llm.invoke([
    HumanMessage(content="Contexto: [lista de documentos recuperados]\n\nPregunta: What is MDT?")
])
print(response)

ResponseError: invalid format: expected "json" or a JSON schema

In [33]:
print(chunks_vector)
print(prompt.invoke({"context": chunks_vector, "question": "What is MDT?"}))


tags=['AzureSearch', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.azuresearch.AzureSearch object at 0x0000021B22FBDD10>
messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is MDT? \nContext: tags=['AzureSearch', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.azuresearch.AzureSearch object at 0x0000021B22FBDD10> \nAnswer:", additional_kwargs={}, response_metadata={})]
